In [27]:
import os

nodes = -1
graph = -1

crtDir =  os.getcwd()
#with open(os.path.join(crtDir, 'fricker26.txt')) as f:
with open(os.path.join(crtDir, 'mediumF.txt')) as f:
    content = f.readlines()

    content = [x.strip() for x in content]
    content = [x.strip(" ") for x in content]

    nodes = int(content[0])

    graph = []

    for i in range(0,nodes):
        graph.append([])
        for j in range(0,nodes):
            graph[i].append(int(str(content[i+1]).split(",")[j]))

In [20]:
import os
import math

nodes = -1
graph = -1

crtDir =  os.getcwd()
#with open(os.path.join(crtDir, 'berlin52.txt')) as f:
with open(os.path.join(crtDir, 'hardE.txt')) as f:
    content = f.readlines()
    
    start = False
    nodes = -1
    coordinates = []
    for line in content:
        if(start == True and line != "EOF\n" and line != "EOF"):
            info = line.strip(" \n")
            info = info.split(" ")
            nodes = int(info[0])
            coordX = float(info[1])
            coordY = float(info[2])
            coordinates.append([coordX,coordY])
        if(line == "NODE_COORD_SECTION\n"):
            start = True
    
    graph = []
    for i in range(0,nodes):
        graph.append([])
        for j in range(0,nodes):
            dist = math.hypot(coordinates[j][0] - coordinates[i][0], coordinates[j][1] - coordinates[i][1])
            graph[i].append(dist)

In [28]:
from random import randint, seed


def generateARandomPermutation(n):
    perm = [i+1 for i in range(n-1)]
    perm.append(0)
    pos1 = randint(0, n - 2)
    pos2 = randint(0, n - 2)
    perm[pos1], perm[pos2] = perm[pos2], perm[pos1]
    return perm

# permutation-based representation
class Chromosome:
    def __init__(self, problParam = None):
        self.__problParam = problParam  #problParam has to store the number of nodes/cities
        self.__repres = generateARandomPermutation(self.__problParam['noNodes'])
        self.__fitness = 0.0
    
    @property
    def repres(self):
        return self.__repres 
    
    @property
    def fitness(self):
        return self.__fitness 
    
    @repres.setter
    def repres(self, l = []):
        self.__repres = l 
    
    @fitness.setter 
    def fitness(self, fit = 0.0):
        self.__fitness = fit 
    
    def crossover(self, c):
        pos1 = randint(0, self.__problParam['noNodes'] - 2)
        pos2 = randint(0, self.__problParam['noNodes'] - 2)
        if (pos2 < pos1):
            pos1, pos2 = pos2, pos1 
        k = 0
        newrepres = self.__repres[pos1 : pos2]
        for el in c.__repres[pos2:] +c.__repres[:pos2]:
            if (el not in newrepres):
                if (len(newrepres) < self.__problParam['noNodes'] - pos1 and el != 0):
                    newrepres.append(el)
                elif (el != 0):
                    newrepres.insert(k, el)
                    k += 1
        newrepres.append(0)
        offspring = Chromosome(self.__problParam)
        offspring.repres = newrepres
        return offspring
    
    def mutation(self):
        pos1 = randint(0, self.__problParam['noNodes'] - 3)
        pos2 = randint(0, self.__problParam['noNodes'] - 3)
        if (pos2 < pos1):
            pos1, pos2 = pos2, pos1
        el = self.__repres[pos2]
        del self.__repres[pos2]
        self.__repres.insert(pos1 + 1, el)
        
    def __str__(self):
        return "\nChromo: " + str(self.__repres) + " has fit: " + str(self.__fitness)
    
    def __repr__(self):
        return self.__str__()
    
    def __eq__(self, c):
        return self.__repres == c.__repres and self.__fitness == c.__fitness

In [29]:
from random import randint

class GA:
    def __init__(self, param = None, problParam = None):
        self.__param = param
        self.__problParam = problParam
        self.__population = []
        
    @property
    def population(self):
        return self.__population
    
    def initialisation(self):
        for _ in range(0, self.__param["popSize"]):
            c = Chromosome(self.__problParam)
            self.__population.append(c)
    
    def evaluation(self):
        for c in self.__population:
            c.fitness = self.__problParam['function'](c.repres)
            
    def bestChromosome(self):
        best = self.__population[0]
        for c in self.__population:
            if (c.fitness < best.fitness):
                best = c
        return best
        
    def worstChromosome(self):
        best = self.__population[0]
        for c in self.__population:
            if (c.fitness > best.fitness):
                best = c
        return best

    def selection(self):
        pos1 = randint(0, self.__param["popSize"] - 1)
        pos2 = randint(0, self.__param["popSize"] - 1)
        if (self.__population[pos1].fitness < self.__population[pos2].fitness):
            return pos1
        else:
            return pos2 
        
    
    def oneGeneration(self):
        newPop = []
        for _ in range(self.__param["popSize"]):
            p1 = self.__population[self.selection()]
            p2 = self.__population[self.selection()]
            off = p1.crossover(p2)
            off.mutation()
            newPop.append(off)
        self.__population = newPop
        self.evaluation()

    def oneGenerationElitism(self):
        newPop = [self.bestChromosome()]
        for _ in range(self.__param["popSize"] - 1):
            p1 = self.__population[self.selection()]
            p2 = self.__population[self.selection()]
            off = p1.crossover(p2)
            off.mutation()
            newPop.append(off)
        self.__population = newPop
        self.evaluation()
        
    def oneGenerationSteadyState(self):
        for _ in range(self.__param["popSize"]):
            p1 = self.__population[self.selection()]
            p2 = self.__population[self.selection()]
            off = p1.crossover(p2)
            off.mutation()
            off.fitness = self.__problParam['function'](off.repres)
            worst = self.worstChromosome()
            if (off.fitness < worst.fitness):
                worst = off

In [30]:
def fcEval(repres):
    s = 0
    sum = 0
    for i in range(0,nodes):
        sum += graph[s][repres[i]]
        s = repres[i]
    return sum

In [31]:
# seed(1)

gaParam = {"popSize": 10, "noGen" : 10000, "pc" : 0.8, "pm" : 0.1}

problParam = {'function' : fcEval, 'noNodes' : nodes}

ga = GA(gaParam, problParam)
ga.initialisation()
ga.evaluation()

stop = False
g = -1
while (not stop and g < gaParam['noGen']):
    g += 1
    #ga.oneGeneration()
    ga.oneGenerationElitism()
    #ga.oneGenerationSteadyState()
    
    bestChromo = ga.bestChromosome()
    print('Best solution in generation ' + str(g) + ' is: x = ' + str(bestChromo.repres))
    print(' f(x) = ' + str(bestChromo.fitness))
    print(" ")

Best solution in generation 0 is: x = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 24, 22, 23, 25, 21, 0]
 f(x) = 1135
 
Best solution in generation 1 is: x = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 24, 22, 23, 25, 21, 0]
 f(x) = 1135
 
Best solution in generation 2 is: x = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 24, 22, 23, 25, 21, 0]
 f(x) = 1135
 
Best solution in generation 3 is: x = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 24, 22, 23, 25, 21, 0]
 f(x) = 1135
 
Best solution in generation 4 is: x = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 24, 22, 23, 25, 21, 0]
 f(x) = 1135
 
Best solution in generation 5 is: x = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 24, 22, 23, 25, 21, 0]
 f(x) = 1135
 
Best solution in generation 6 is: x = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 24

Best solution in generation 477 is: x = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 13, 15, 16, 17, 18, 19, 20, 24, 22, 23, 25, 21, 0]
 f(x) = 1125
 
Best solution in generation 478 is: x = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 13, 15, 16, 17, 18, 19, 20, 24, 22, 23, 25, 21, 0]
 f(x) = 1125
 
Best solution in generation 479 is: x = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 13, 15, 16, 17, 18, 19, 20, 24, 22, 23, 25, 21, 0]
 f(x) = 1125
 
Best solution in generation 480 is: x = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 13, 15, 16, 17, 18, 19, 20, 24, 22, 23, 25, 21, 0]
 f(x) = 1125
 
Best solution in generation 481 is: x = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 13, 15, 16, 17, 18, 19, 20, 24, 22, 23, 25, 21, 0]
 f(x) = 1125
 
Best solution in generation 482 is: x = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 13, 15, 16, 17, 18, 19, 20, 24, 22, 23, 25, 21, 0]
 f(x) = 1125
 
Best solution in generation 483 is: x = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 13, 15, 16, 17, 

Best solution in generation 841 is: x = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 13, 15, 16, 17, 18, 19, 20, 24, 22, 23, 25, 21, 0]
 f(x) = 1125
 
Best solution in generation 842 is: x = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 13, 15, 16, 17, 18, 19, 20, 24, 22, 23, 25, 21, 0]
 f(x) = 1125
 
Best solution in generation 843 is: x = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 13, 15, 16, 17, 18, 19, 20, 24, 22, 23, 25, 21, 0]
 f(x) = 1125
 
Best solution in generation 844 is: x = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 13, 15, 16, 17, 18, 19, 20, 24, 22, 23, 25, 21, 0]
 f(x) = 1125
 
Best solution in generation 845 is: x = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 13, 15, 16, 17, 18, 19, 20, 24, 22, 23, 25, 21, 0]
 f(x) = 1125
 
Best solution in generation 846 is: x = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 13, 15, 16, 17, 18, 19, 20, 24, 22, 23, 25, 21, 0]
 f(x) = 1125
 
Best solution in generation 847 is: x = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 13, 15, 16, 17, 

Best solution in generation 1310 is: x = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 13, 15, 16, 17, 19, 18, 20, 24, 22, 23, 25, 21, 0]
 f(x) = 1116
 
Best solution in generation 1311 is: x = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 13, 15, 16, 17, 19, 18, 20, 24, 22, 23, 25, 21, 0]
 f(x) = 1116
 
Best solution in generation 1312 is: x = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 13, 15, 16, 17, 19, 18, 20, 24, 22, 23, 25, 21, 0]
 f(x) = 1116
 
Best solution in generation 1313 is: x = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 13, 15, 16, 17, 19, 18, 20, 24, 22, 23, 25, 21, 0]
 f(x) = 1116
 
Best solution in generation 1314 is: x = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 13, 15, 16, 17, 19, 18, 20, 24, 22, 23, 25, 21, 0]
 f(x) = 1116
 
Best solution in generation 1315 is: x = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 13, 15, 16, 17, 19, 18, 20, 24, 22, 23, 25, 21, 0]
 f(x) = 1116
 
Best solution in generation 1316 is: x = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 13, 15, 1

 
Best solution in generation 1698 is: x = [1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 10, 12, 14, 13, 15, 16, 17, 19, 18, 20, 22, 23, 25, 21, 24, 0]
 f(x) = 1100
 
Best solution in generation 1699 is: x = [1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 10, 12, 14, 13, 15, 16, 17, 19, 18, 20, 22, 23, 25, 21, 24, 0]
 f(x) = 1100
 
Best solution in generation 1700 is: x = [1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 10, 12, 14, 13, 15, 16, 17, 19, 18, 20, 22, 23, 25, 21, 24, 0]
 f(x) = 1100
 
Best solution in generation 1701 is: x = [1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 10, 12, 14, 13, 15, 16, 17, 19, 18, 20, 22, 23, 25, 21, 24, 0]
 f(x) = 1100
 
Best solution in generation 1702 is: x = [1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 10, 12, 14, 13, 15, 16, 17, 19, 18, 20, 22, 23, 25, 21, 24, 0]
 f(x) = 1100
 
Best solution in generation 1703 is: x = [1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 10, 12, 14, 13, 15, 16, 17, 19, 18, 20, 22, 23, 25, 21, 24, 0]
 f(x) = 1100
 
Best solution in generation 1704 is: x = [1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 10, 12, 14, 13, 15,

...

 
Best solution in generation 8317 is: x = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 11, 14, 13, 15, 16, 17, 19, 18, 20, 21, 25, 22, 23, 24, 0]
 f(x) = 1012
 
Best solution in generation 8318 is: x = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 11, 14, 13, 15, 16, 17, 19, 18, 20, 21, 25, 22, 23, 24, 0]
 f(x) = 1012
 
Best solution in generation 8319 is: x = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 11, 14, 13, 15, 16, 17, 19, 18, 20, 21, 25, 22, 23, 24, 0]
 f(x) = 1012
 
Best solution in generation 8320 is: x = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 11, 14, 13, 15, 16, 17, 19, 18, 20, 21, 25, 22, 23, 24, 0]
 f(x) = 1012
 
Best solution in generation 8321 is: x = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 11, 14, 13, 15, 16, 17, 19, 18, 20, 21, 25, 22, 23, 24, 0]
 f(x) = 1012
 
Best solution in generation 8322 is: x = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 11, 14, 13, 15, 16, 17, 19, 18, 20, 21, 25, 22, 23, 24, 0]
 f(x) = 1012
 
Best solution in generation 8323 is: x = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 11, 14, 13, 15,

 f(x) = 1012
 
Best solution in generation 8808 is: x = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 11, 14, 13, 15, 16, 17, 19, 18, 20, 21, 25, 22, 23, 24, 0]
 f(x) = 1012
 
Best solution in generation 8809 is: x = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 11, 14, 13, 15, 16, 17, 19, 18, 20, 21, 25, 22, 23, 24, 0]
 f(x) = 1012
 
Best solution in generation 8810 is: x = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 11, 14, 13, 15, 16, 17, 19, 18, 20, 21, 25, 22, 23, 24, 0]
 f(x) = 1012
 
Best solution in generation 8811 is: x = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 11, 14, 13, 15, 16, 17, 19, 18, 20, 21, 25, 22, 23, 24, 0]
 f(x) = 1012
 
Best solution in generation 8812 is: x = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 11, 14, 13, 15, 16, 17, 19, 18, 20, 21, 25, 22, 23, 24, 0]
 f(x) = 1012
 
Best solution in generation 8813 is: x = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 11, 14, 13, 15, 16, 17, 19, 18, 20, 21, 25, 22, 23, 24, 0]
 f(x) = 1012
 
Best solution in generation 8814 is: x = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 11

 
Best solution in generation 9233 is: x = [1, 2, 3, 5, 4, 6, 7, 8, 9, 10, 12, 11, 14, 13, 15, 16, 17, 19, 18, 20, 21, 25, 22, 23, 24, 0]
 f(x) = 1008
 
Best solution in generation 9234 is: x = [1, 2, 3, 5, 4, 6, 7, 8, 9, 10, 12, 11, 14, 13, 15, 16, 17, 19, 18, 20, 21, 25, 22, 23, 24, 0]
 f(x) = 1008
 
Best solution in generation 9235 is: x = [1, 2, 3, 5, 4, 6, 7, 8, 9, 10, 12, 11, 14, 13, 15, 16, 17, 19, 18, 20, 21, 25, 22, 23, 24, 0]
 f(x) = 1008
 
Best solution in generation 9236 is: x = [1, 2, 3, 5, 4, 6, 7, 8, 9, 10, 12, 11, 14, 13, 15, 16, 17, 19, 18, 20, 21, 25, 22, 23, 24, 0]
 f(x) = 1008
 
Best solution in generation 9237 is: x = [1, 2, 3, 5, 4, 6, 7, 8, 9, 10, 12, 11, 14, 13, 15, 16, 17, 19, 18, 20, 21, 25, 22, 23, 24, 0]
 f(x) = 1008
 
Best solution in generation 9238 is: x = [1, 2, 3, 5, 4, 6, 7, 8, 9, 10, 12, 11, 14, 13, 15, 16, 17, 19, 18, 20, 21, 25, 22, 23, 24, 0]
 f(x) = 1008
 
Best solution in generation 9239 is: x = [1, 2, 3, 5, 4, 6, 7, 8, 9, 10, 12, 11, 14, 13, 15,

Best solution in generation 9514 is: x = [1, 2, 3, 5, 4, 6, 7, 8, 9, 10, 12, 11, 14, 13, 15, 16, 17, 19, 18, 20, 21, 25, 22, 23, 24, 0]
 f(x) = 1008
 
Best solution in generation 9515 is: x = [1, 2, 3, 5, 4, 6, 7, 8, 9, 10, 12, 11, 14, 13, 15, 16, 17, 19, 18, 20, 21, 25, 22, 23, 24, 0]
 f(x) = 1008
 
Best solution in generation 9516 is: x = [1, 2, 3, 5, 4, 6, 7, 8, 9, 10, 12, 11, 14, 13, 15, 16, 17, 19, 18, 20, 21, 25, 22, 23, 24, 0]
 f(x) = 1008
 
Best solution in generation 9517 is: x = [1, 2, 3, 5, 4, 6, 7, 8, 9, 10, 12, 11, 14, 13, 15, 16, 17, 19, 18, 20, 21, 25, 22, 23, 24, 0]
 f(x) = 1008
 
Best solution in generation 9518 is: x = [1, 2, 3, 5, 4, 6, 7, 8, 9, 10, 12, 11, 14, 13, 15, 16, 17, 19, 18, 20, 21, 25, 22, 23, 24, 0]
 f(x) = 1008
 
Best solution in generation 9519 is: x = [1, 2, 3, 5, 4, 6, 7, 8, 9, 10, 12, 11, 14, 13, 15, 16, 17, 19, 18, 20, 21, 25, 22, 23, 24, 0]
 f(x) = 1008
 
Best solution in generation 9520 is: x = [1, 2, 3, 5, 4, 6, 7, 8, 9, 10, 12, 11, 14, 13, 15, 1

Best solution in generation 9808 is: x = [1, 2, 3, 5, 4, 6, 7, 8, 9, 10, 12, 11, 14, 13, 15, 16, 17, 19, 18, 20, 21, 25, 22, 23, 24, 0]
 f(x) = 1008
 
Best solution in generation 9809 is: x = [1, 2, 3, 5, 4, 6, 7, 8, 9, 10, 12, 11, 14, 13, 15, 16, 17, 19, 18, 20, 21, 25, 22, 23, 24, 0]
 f(x) = 1008
 
Best solution in generation 9810 is: x = [1, 2, 3, 5, 4, 6, 7, 8, 9, 10, 12, 11, 14, 13, 15, 16, 17, 19, 18, 20, 21, 25, 22, 23, 24, 0]
 f(x) = 1008
 
Best solution in generation 9811 is: x = [1, 2, 3, 5, 4, 6, 7, 8, 9, 10, 12, 11, 14, 13, 15, 16, 17, 19, 18, 20, 21, 25, 22, 23, 24, 0]
 f(x) = 1008
 
Best solution in generation 9812 is: x = [1, 2, 3, 5, 4, 6, 7, 8, 9, 10, 12, 11, 14, 13, 15, 16, 17, 19, 18, 20, 21, 25, 22, 23, 24, 0]
 f(x) = 1008
 
Best solution in generation 9813 is: x = [1, 2, 3, 5, 4, 6, 7, 8, 9, 10, 12, 11, 14, 13, 15, 16, 17, 19, 18, 20, 21, 25, 22, 23, 24, 0]
 f(x) = 1008
 
Best solution in generation 9814 is: x = [1, 2, 3, 5, 4, 6, 7, 8, 9, 10, 12, 11, 14, 13, 15, 1